In [1]:
import copy
import sqlite3
import numpy as np
import scipy.optimize as optim

class SQLDatagenerator:
  
      """
        Code from https://github.com/baireuther/neural_network_decoder/tree/arXiv1705p07855
      """
      def __init__(self, trainFile, validationFile, testFile):
        self.dim_syndr = 8
        self.n_steps_net2 = 4
        
        self.load_data(trainFile, validationFile, testFile)
        
      # # # functions that load the databases and generate batches # # #
      def load_data(self, training_fname, validation_fname, test_fname):
        """ this function loads the training, validation, and test databases

        Input
        -----

        training_fname -- path + filename of the training database
        valiation_fname -- path + filename of the validation database
        test_fname -- path + filename of the test database
        store_in_RAM -- if True the databases will be stored in the RAM
        """

        self.training_conn = sqlite3.connect(training_fname)
        self.validation_conn = sqlite3.connect(validation_fname)
        self.test_conn = sqlite3.connect(test_fname)

        training_c = self.training_conn.cursor()
        validation_c = self.validation_conn.cursor()
        test_c = self.test_conn.cursor()

        # get all the seeds
        training_c.execute('SELECT seed FROM data')
        validation_c.execute('SELECT seed FROM data')
        test_c.execute('SELECT seed FROM data')
        
        self.training_keys = list(sorted([s[0] for s in training_c.fetchall()]))
        self.validation_keys = list(sorted([s[0] for s in validation_c.fetchall()]))
        self.test_keys = list(sorted([s[0] for s in test_c.fetchall()]))

        # checks that there is no overlapp in the seeds of the data sets
        N_training = len(self.training_keys)
        N_validation = len(self.validation_keys)
        N_test = len(self.test_keys)
        all_keys = set(self.training_keys + self.validation_keys + self.test_keys)
        
        if len(all_keys) < N_training + N_validation + N_test:
          raise ValueError("There is overlapp between the seeds of the"
                           " training,  validation, and test sets. This"
                           "is bad practice")
        print("loaded databases and checked exclusiveness training, "
              "validation, and test keys")

        print("N_training=" + str(N_training) + ", N_validaiton=" +
              str(N_validation) + ", N_test=" + str(N_test) + ".")

      def close_databases(self):
        """ This function closes all databases """
        self.training_conn.close()
        self.validation_conn.close()
        self.test_conn.close()

      def gen_batch(self, sample):
        """ formats a single batch of data

        Input
        -----

        sample - raw data from the database
        """

        syndr, fsyndr, parity, length = sample
        n_steps = int(len(syndr) / self.dim_syndr)

        # format into shape [steps, syndromes]
        syndr1 = np.fromstring(syndr, dtype=bool).reshape([n_steps, -1])

        # get and set length information
        len1 = np.frombuffer(length, dtype=int)[0]

        # the second length is set by n_steps_net2, except if len1 is shorter
        len2 = min(len1, self.n_steps_net2)

        syndr2 = syndr1[len1 - len2:len1 - len2 + self.n_steps_net2]
        fsyndr = np.fromstring(fsyndr, dtype=bool)
        parity = np.frombuffer(parity, dtype=bool)

        return syndr1, syndr2, fsyndr, len1, len2, parity

      def gen_batch_oversample(self, sample, max_steps=None):
        """ formats a single batch of data with final syndrome increments
            at each time steps into multiple batches with a signle final
            syndrome increment

        Input
        -----

        sample - raw data from the database
        max_steps -- maximum number of steps for oversampling
        """

        syndrs, fsyndrs, parities = sample
        max_steps = min([len(parities), max_steps])

        syndrs = np.fromstring(syndrs, dtype=bool).reshape([len(parities), -1])
        fsyndrs = np.fromstring(
            fsyndrs, dtype=bool).reshape([len(parities), -1])
        parities = np.frombuffer(parities, dtype=bool)

        syndr1_l, syndr2_l, fsyndr_l, len1_l, len2_l, parity_l \
            = [], [], [], [], [], []

        step_list = []
        stepsize, step = 1, 1
        for n in range(self.n_steps_net2, max_steps + 1):
          if np.mod(step, stepsize) == stepsize - 1:
            step_list.append(n)
            stepsize += 1
            step = 0
          else:
            step += 1

        for n in step_list:
          if n <= max_steps:

            # format into shape [steps, syndromes]
            syndr1 = np.concatenate(
                (syndrs[:n], np.zeros((max_steps - n, self.dim_syndr),
                                      dtype=bool)), axis=0)
            syndr1_l.append(syndr1)

            # get and set length information
            len1 = n
            len1_l.append(len1)

            # the second length is set by n_steps_net2, except if len1 is
            # shorter
            len2 = min(len1, self.n_steps_net2)
            len2_l.append(len2)

            syndr2_l.append(
                syndr1[len1 - len2:len1 - len2 + self.n_steps_net2])
            fsyndr_l.append(fsyndrs[n - 1])
            parity_l.append(parities[n - 1])

        return syndr1_l, syndr2_l, fsyndr_l, len1_l, len2_l, parity_l

      def gen_batches(self, batch_size, n_batches, data_type, oversample=False, max_steps=None):
        """ a genererator to generate the training, validation, and test batches

        Input
        -----

        batch_size -- number of samples per batch
        n_batches -- number of batches
        data_type -- 'training', 'validation', or 'test'  data

        Output
        ------

        a generator containing formatted batches of:
        syndrome increments for the first network, for the second network,
        the final syndrome increments, length information for the first
        network, for the second network, and the final parities
        """

        # select data from the corresponding database
        if data_type == "training":
          c = self.training_conn.cursor()
        elif data_type == "validation":
          c = self.validation_conn.cursor()
        elif data_type == "test":
          c = self.test_conn.cursor()
        else:
          raise ValueError("The only allowed data_types are: 'training','validation' and 'test'.")
        
        
        if oversample:
          c.execute("SELECT events, err_signal, parities FROM data ORDER BY RANDOM() LIMIT ?",
                    (n_batches * batch_size, ))
        else:
          c.execute("SELECT events, err_signal, parity, length FROM data ORDER BY RANDOM() LIMIT ?",
                    (n_batches * batch_size, ))

        for n in range(n_batches):

          arrX1, arrX2, arrFX, arrL1, arrL2, arrY = [], [], [], [], [], []
          samples = c.fetchmany(batch_size)

          for sample in samples:
            if oversample:
              syndr1_l, syndr2_l, fsyndr_l, len1_l, len2_l, parity_l \
                  = self.gen_batch_oversample(sample, max_steps)

              for syndr1, syndr2, fsyndr, len1, len2, parity in \
                      zip(syndr1_l, syndr2_l, fsyndr_l, len1_l, len2_l, parity_l):
                arrX1.append(syndr1)
                arrX2.append(syndr2)
                arrFX.append(fsyndr)
                arrL1.append(len1)
                arrL2.append(len2)
                arrY.append(parity)
            else:
              syndr1, syndr2, fsyndr, len1, len2, parity \
                  = self.gen_batch(sample)

              arrX1.append(syndr1)
              arrX2.append(syndr2)
              arrFX.append(fsyndr)
              arrL1.append(len1)
              arrL2.append(len2)
              arrY.append(parity)

          arrX1 = np.array(arrX1)
          arrX2 = np.array(arrX2)
          arrFX = np.array(arrFX)
          arrL1 = np.array(arrL1)
          arrL2 = np.array(arrL2)
          arrY = np.array(arrY)

          yield arrX1, arrX2, arrFX, arrL1, arrL2, arrY
          
        
        

In [12]:
# path of databases (must exist)
db_path = "./data/"

# filenames of databases (this must be sqlite3 databases)
train_fname = "surf17_small_train.db"
validation_fname = "surf17_small_validation.db"
test_fname = "surf17_small_test.db"

gen = SQLDatagenerator(db_path + train_fname,
             db_path + validation_fname,
             db_path + test_fname)

n_batches = 5
batch_size = 200
trainingGenerator = gen.gen_batches(batch_size=batch_size, 
                                 n_batches=n_batches,
                                 data_type='training')

validationGenerator = gen.gen_batches(batch_size=batch_size, 
                                      n_batches=n_batches,
                                      data_type='validation')

testGenerator = gen.gen_batches(batch_size=batch_size, 
                                n_batches=n_batches,
                                data_type='test')

X_train_syndr = [] # Syndromes for the first layer
X_train_syndr2 = [] # Syndromes for the second layer
X_train_fsyndr = [] # Final syndromes
y_train = [] # Final parity measurement

for k in range(n_batches):
  print("Batch no. {0}".format(k))
  x1, x2, fx, l1, l2, y = next(trainingGenerator)
  X_train_syndr.append(x1)
  X_train_syndr2.append(x2)
  X_train_fsyndr.append(fx)
  y_train.append(y)

# Normalize data
for k in range(len(X_train_syndr)):
  X_train_syndr[k] = X_train_syndr[k].reshape((X_train_syndr[k].shape[0],
                                               X_train_syndr[k].shape[1]*X_train_syndr[k].shape[2]))
  
  X_train_syndr2[k] = X_train_syndr2[k].reshape((X_train_syndr2[k].shape[0],
                                                 X_train_syndr2[k].shape[1]*X_train_syndr2[k].shape[2]))

X_train_syndr = np.array(X_train_syndr, dtype=int)
X_train_syndr2 = np.array(X_train_syndr2, dtype=int)
X_train_fsyndr = np.array(X_train_fsyndr, dtype=int)
y_train = np.array(y_train, dtype=int)

# for k in range(1):
#   print("Step={}".format(k))
#   x1, x2, fx, l1, l2, y = next(trainGenerator)
#   [print(x.shape) for x in x1]
#   [print(x.shape) for x in x2]
#   [print(yy.shape) for yy in y]
#   print("x1.shape={0}".format(x1.shape))
#   print("x2.shape={0}".format(x2.shape))
#   print("fx.shape={0}".format(fx.shape))
#   print("l1.shape={0}".format(l1.shape))
#   print("l2.shape={0}".format(l2.shape))
#   print("y.shape={0}".format(y.shape)) 
#   print("l1={0}".format(l1))
#   print("l2={0}".format(l2))

# X_train_syndr = X_train_syndr.reshape((X_train_syndr.shape[0], X_train_syndr.shape[1],
#                                       X_train_syndr.shape[2]*X_train_syndr.shape[3]))

# X_train_syndr2 = X_train_syndr2.reshape((X_train_syndr2.shape[0], X_train_syndr2.shape[1],
#                                       X_train_syndr2.shape[2]*X_train_syndr2.shape[3]))

# X_train_fsyndr = X_train_fsyndr.reshape((X_train_fsyndr.shape[0]*X_train_fsyndr.shape[1],
#                                       X_train_fsyndr.shape[2]))

# y_train = y_train.reshape((y_train.shape[0]*y_train.shape[1],
#                                       y_train.shape[2]))

print("X_train_syndr.shape={0}".format(X_train_syndr.shape))
print("X_train_syndr2.shape={0}".format(X_train_syndr2.shape))
print("X_train_fsyndr.shape={0}".format(X_train_fsyndr.shape))
print("y_train.shape={0}".format(y_train.shape))

loaded databases and checked exclusiveness training, validation, and test keys
N_training=1000, N_validaiton=100, N_test=200.
Batch no. 0
Batch no. 1
Batch no. 2
Batch no. 3
Batch no. 4
X_train_syndr.shape=(5, 200, 160)
X_train_syndr2.shape=(5, 200, 32)
X_train_fsyndr.shape=(5, 200, 4)
y_train.shape=(5, 200, 1)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:91: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [13]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, LSTM

# This returns a tensor
input_syndr = Input(shape=(X_train_syndr[0].shape))

# a layer instance is callable on a tensor, and returns a tensor
hidden_size=64
x = LSTM(hidden_size, return_sequences=True)(input_syndr)
x = LSTM(hidden_size, return_sequences=True)(x)
x = Dense(hidden_size, activation='relu')(x)
predictions = Dense(1, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=input_syndr, outputs=predictions)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
#model.fit(X_train_syndr[0], y_train[0])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200, 160)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 200, 64)           57600     
_________________________________________________________________
lstm_7 (LSTM)                (None, 200, 64)           33024     
_________________________________________________________________
dense_5 (Dense)              (None, 200, 64)           4160      
_________________________________________________________________
dense_6 (Dense)              (None, 200, 1)            65        
Total params: 94,849
Trainable params: 94,849
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train_syndr, y_train)

Epoch 1/1
5/5 [==============================] - 2s 445ms/step - loss: 9.5973 - acc: 0.3980


In [5]:
print(X_train_syndr.shape)

(5, 4000, 8)


In [6]:
print(y_train.shape)

(5, 200, 1)


In [ ]:
Csinalj 1 db lstm layert, resahpe all batches, batchonkent fittelni, es csak a szimpla syndrome increment, final syndrome nalkul

In [ ]:
import keras
keras.__version__

import Number

In [ ]:
th = Number.Three()

In [ ]:
th.getThree()